In [1]:
import os
from crewai import Agent, Task, Crew, Process, LLM  
from crewai_tools import SerperDevTool
from dotenv import load_dotenv

c:\Users\adesh\anaconda3\envs\agent\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\adesh\anaconda3\envs\agent\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\adesh\anaconda3\envs\agent\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You sh

In [2]:
load_dotenv()

os.environ["GEMINI_API_KEY"]=os.getenv('GEMINI_API_KEY')
os.environ["SERPER_API_KEY"]=os.getenv('SERPER_API_KEY')
# os.environ["GEMINI_API_KEY"]= gemini_key
# os.environ["SERPER_API_KEY"] = serper_key

In [3]:
# Initialize a search tool (to fetch real-time travel info)
search_tool = SerperDevTool()

# Define the AI Model
llm = LLM(model="gemini/gemini-1.5-flash", # model name
          verbose=True, 
          temperature=0.5,
          api_key=os.environ["GEMINI_API_KEY"])

In [ ]:
researcher = Agent(
    role="Social Media Content Researcher",
    goal="""Conduct in-depth research on the topic "{topic}" for creating high-quality social media content for "{platform}".""",
    backstory="""
    You are an expert social media researcher, skilled in gathering **reliable** and **verified** content from the internet. 
    Your primary responsibility is to ensure that all collected information, **URLs, and image links** are **valid, accurate, and functional**. 
    You personally verify each URL and cross-check its relevance to the topic before passing the information to post creators.
    """,
    verbose=True,
    memory=True,
    llm=llm,
    tools=[search_tool],
    allow_delegation=True,
)

facebook_agent = Agent(
    role="Facebook Content Creator",
    goal="Generate engaging and **algorithm-friendly** Facebook posts based on researched content.",
    backstory="""
    You specialize in crafting **highly shareable** Facebook content. 
    You understand Facebook’s content strategies and **optimize posts for engagement** through captions, images, and hashtags.
    """,
    verbose=True,
    memory=True,
    llm=llm,
    tools=[search_tool],
    allow_delegation=False,
)

instagram_agent = Agent(
    role="Instagram Content Creator",
    goal="Create visually appealing and **aesthetically optimized** Instagram posts, including engaging captions and hashtags.",
    backstory="""
    You are a **creative Instagram expert**, skilled in curating visually appealing posts with **impactful storytelling, creative captions, and well-researched hashtags**.
    """,
    verbose=True,
    memory=True,
    llm=llm,
    tools=[search_tool],
    allow_delegation=False,
)

twitter_agent = Agent(
    role="Twitter Content Creator",
    goal="Craft short, **concise**, and engaging tweets based on researched content.",
    backstory="You are an expert in creating **highly shareable** and **engagement-driven** tweets with strategic use of hashtags and media.",
    verbose=True,
    memory=True,
    llm=llm,
    tools=[search_tool],
    allow_delegation=False,
)

linkedin_agent = Agent(
    role="LinkedIn Content Creator",
    goal="Generate **professional and insightful** LinkedIn posts based on researched content.",
    backstory="You are a LinkedIn **business content expert**, specializing in **thought leadership posts, industry insights, and professional storytelling**.",
    verbose=True,
    memory=True,
    llm=llm,
    tools=[search_tool],
    allow_delegation=False,
)

manager = Agent(
    role="Social Media Project Manager",
    goal="""
    Efficiently manage the research and post creation workflow for {platform}.
    - Assign ONLY the most relevant agent based on "{platform}".
    - Do NOT attempt to use multiple social media agents for one task.
    - Trust the researcher's findings and verify content **only if necessary**.
    - If an image is unavailable or broken, instruct the researcher to select another from the remaining top 10 results.
    - Avoid unnecessary retries or micromanaging agents.
    """,
    backstory="""
    You are an experienced AI-powered manager skilled at overseeing research and content creation.
    Your job is to **manage agents efficiently, not interfere unnecessarily**.
    You assign the correct agent, review their work briefly, and move forward.
    """,
    verbose=True,
    memory=True,
    llm=llm,
    allow_delegation=True,
)

In [ ]:
research_task = Task(
    description="""
    Conduct research on the topic "{topic}" for a social media post on {platform}.
    - Gather **accurate and relevant** information.
    - Find **ONE best image URL** from the **top 10 Google Image search results**.
    - If the first image is broken, return another from the remaining top 10.
    - Ensure the final research contains **verified, useful external URLs**.
    """,
    expected_output="""
    Research should return:
      1. **High-quality content**.
      2. **One best image URL** (directly from top 10 Google Images).
      3. **A concise, engaging caption**.
      4. **Relevant, trending hashtags**.
      5. **Verified external URLs** (if applicable).
    """,
    agent=researcher,
    tools=[search_tool],
)

facebook_task = Task(
    description="Create a **Facebook post** optimized for engagement, ensuring that the **text, images, and hashtags** align with Facebook’s best practices.",
    expected_output="""
    A **high-quality Facebook post** that includes:
      - Well-structured **text content**.
      - **Image (if available from research)**.
      - Relevant **hashtags** (if applicable).
      - A **clear call to action** (if relevant).
    """,
    tools=[search_tool],
    agent=facebook_agent,
)

instagram_task = Task(
    description="Generate an **Instagram post** that is **visually appealing**, engaging, and **caption-optimized**.",
    expected_output="""
    A **well-crafted Instagram post** that includes:
      - A **compelling caption** related to the topic.
      - **Mandatory image** (from research).
      - A well-researched set of **hashtags**.
    """,
    tools=[search_tool],
    agent=instagram_agent,
)

twitter_task = Task(
    description="Generate a Twitter post based on researched content.",
    expected_output="""
    A **high-impact tweet** that includes:
      - A **concise but engaging message** (under 280 characters).
      - Optional **image (if available from research)**.
      - **Relevant hashtags** (if applicable).
    """,
    tools=[search_tool],
    agent=twitter_agent,
)

linkedin_task = Task(
    description="Generate a **professional LinkedIn post** that provides value and encourages engagement.",
    expected_output="""
    A **thoughtful and professional LinkedIn post** including:
      - **Industry-relevant, insightful content**.
      - Optional **image (if available from research)**.
      - **A structured, easy-to-read format**.
      - Relevant **hashtags** (if applicable).
    """,
    tools=[search_tool],
    agent=linkedin_agent,
)

In [11]:
crew = Crew(
    agents=[researcher, facebook_agent, instagram_agent, twitter_agent, linkedin_agent],
    tasks=[research_task, facebook_task, instagram_task, twitter_task, linkedin_task],
    process=Process.hierarchical,
    manager_agent=manager
)

result = crew.kickoff(inputs={'topic': 'Kerala Tourism', 'platform': 'Instagram'})
print(result)

2025-03-11 13:18:44,283 - 1332 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Social Media Project Manager
## Task: 
    Conduct research on the topic "Kerala Tourism" for a social media post on Instagram.  
    - Gather the most **relevant & accurate** information.  
    - Find **one highly relevant image URL** related to the topic.  
      - **The image can come from any reliable source.**  
      - **No restrictions on where the image is selected from.**  
    - Include **only working URLs** related to the topic.
    
# Agent: Social Media Content Researcher
## Task: Conduct research on Kerala Tourism for an Instagram post.  Gather relevant and accurate information, find one highly relevant image URL from a reliable source (include only working URLs), and suggest a short, engaging caption, relevant hashtags, and useful external URLs (if applicable, only verified).


# Agent: Social Media Content Researcher
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Kerala tourism stunning images\"}"
## Tool Output: 
{'searchP

In [12]:
with open("post.md", "w", encoding='utf-8') as f:
    f.write(result.raw)